In [20]:
# ORB 特征及 bf matcher
import numpy as np 
import cv2 
import matplotlib.pyplot as plt

img1 = cv2.imread("assets/box.png", 0)
img2 = cv2.imread("assets/box_in_scene.png", 0)

# 创建 ORB 对象
orb = cv2.ORB_create()

# 检测关键点并计算特征
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

# 创建匹配器
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# 匹配
matchs = bf.match(des1, des2)

# 排序
matchs = sorted(matchs, key=lambda x: x.distance)

# 绘制匹配结果
img_res = cv2.drawMatches(img1, kp1, img2, kp2, matchs[:15], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2.imshow("orb-bfmatches", img_res)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
# SIFT 和 bf knn matcher
import numpy as np 
import cv2

img1 = cv2.imread("assets/box.png", 0)
img2 = cv2.imread("assets/box_in_scene.png", 0)

sift = cv2.SIFT_create()

kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

good = []
for m, n in matches:
    if m.distance < 0.5*n.distance:
        good.append([m])

img_res = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good,
                            None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2.imshow("sift-bfknnmatches", img_res)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
# sift FLANN matcher
import numpy as np 
import cv2 

img1 = cv2.imread("assets/box.png", 0)
img2 = cv2.imread("assets/box_in_scene.png", 0)

# 初始化创建
sift = cv2.SIFT_create()

# 关键点检测和特征计算
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0, 0] for i in range(len(matches))]
# ratio test as per Lowe's paper
for i, (m, n) in enumerate(matches):
    if m.distance < 0.5*n.distance:
        matchesMask[i] = [1, 0]
draw_params = dict(matchColor=(0, 255, 0),
                    singlePointColor=(255, 0, 0),
                    matchesMask=matchesMask,
                    flags=cv2.DrawMatchesFlags_DEFAULT)
img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matches, None, **draw_params)

cv2.imshow("sift-flannmatches", img3)
cv2.waitKey(0)
cv2.destroyAllWindows()